# Random Shaking Volume Denoising (RSVD) using opticalflow3d

In [ ]:
local_debug = True

In [ ]:
jupyter_server = True

In [ ]:
vol_filename = "epfl1_subset1_float.mrc"

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    print("Running in Colab")
    !pip install cupy-cuda12x
    !pip install opticalflow3D
    !apt install libcudart11.0
    !apt install libcublas11
    !apt install libcufft10
    !apt install libcusparse11
    !apt install libnvrtc11.2
    #from google.colab import drive
    #drive.mount('/content/drive')
    #!cp drive/Shareddrives/TomogramDenoising/tomograms/{vol_name}.tif .
else:
    print("Running in locahost")
    #!cp ~/Downloads/{vol_name}.tif .

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
import opticalflow3D
import warnings
from numba.core.errors import NumbaPerformanceWarning
import numpy as np
import logging
import mrcfile
warnings.filterwarnings("ignore", category=NumbaPerformanceWarning)

In [ ]:
if local_debug:
    !ln -sf ../../motion_estimation/src/motion_estimation .
else:
    !pip install "denoising @ git+https://github.com/vicente-gonzalez-ruiz/motion_estimation"

In [ ]:
if local_debug:
    !ln -sf denoising/volume/RSVD.py .
else:
    !pip install "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
from RSVD import Random_Shaking_Denoising

In [ ]:
if local_debug:
    !ln -sf ../../information_theory/src/information_theory/ .
else:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
import information_theory  # pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
from collections import namedtuple
Args = namedtuple("args", ["input", "output"])
fn, fe = vol_filename.split(".")
args = Args(vol_filename , fn + "_denoised." + fe)
print(args)

In [ ]:
%%bash -s "$args.input"
set -x
OUTPUT_FILENAME=$1
#rm -f $OUTPUT_FILENAME
if test ! -f $OUTPUT_FILENAME ; then
    FILEID="1qe7d9dOJg1H3xlGy1DIgEdAnwuAgA1iB" # https://drive.google.com/file/d/1qe7d9dOJg1H3xlGy1DIgEdAnwuAgA1iB/view?usp=drive_link
    #FILEID="1iui018SGKa5nb0ybeoUAB7uCUhl9EJ5o" #https://drive.google.com/file/d/1iui018SGKa5nb0ybeoUAB7uCUhl9EJ5o/view?usp=sharing
    #wget --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O $OUTPUT_FILENAME #2> /dev/null
    #wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=FILEID" -O FILENAME && rm -rf /tmp/cookies.txt
    ~/envs/OF3D/bin/gdown https://drive.google.com/uc?id=$FILEID # pip install gdown
fi
ls -l $OUTPUT_FILENAME
set +x

In [ ]:
#noisy = opticalflow3D.helpers.load_image(args.input)
stack_MRC = mrcfile.open(args.input)
noisy = stack_MRC.data

In [ ]:
#noisy = (255*(noisy - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.uint8)

In [ ]:
#noisy = noisy[80:, 200:, 200:]

In [ ]:
noisy.shape

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(16, 16))
axs.imshow(noisy[:, ::-1, :][75], cmap="gray")
axs.set_title(f"Noisy min={np.min(noisy):5.2f} max={np.max(noisy):5.2f} avg={np.average(noisy):5.2f}")
fig.tight_layout()
plt.show()

In [ ]:
block_size = (noisy.shape[0], noisy.shape[1], noisy.shape[2])

In [ ]:
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=7, presmoothing=3, filter_type="gaussian", filter_size=7,); RS_sigma = 1.0
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=2, scale=0.5, spatial_size=5, sigma_k=1.0, filter_type="gaussian", filter_size=9, presmoothing=None, device_id=0); RS_sigma = 1.25; N_iters=100
#denoiser = RSVD.Random_Shaking_Denoising(logging_level=logging.INFO)
#denoiser = Random_Shaking_Denoising(logging_level=logging.DEBUG)

def show_image(denoised_volume, title):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title)
    #axs.imshow(denoised_volume[denoised_volume.shape[0]//2].astype(np.uint8), cmap="gray")
    axs.imshow(denoised_volume[denoised_volume.shape[0]//2], cmap="gray")
    plt.show()

def get_quality(noisy, denoised):
    slice_idx = noisy.shape[0]//2
    return information_theory.information.compute_quality_index(noisy[slice_idx], denoised[slice_idx])

denoiser = Random_Shaking_Denoising(logging_level=logging.INFO, show_image=None, get_quality=get_quality)

In [ ]:
"""Farneback3D class used to instantiate the algorithm with its parameters.

Args:
    iters (int): number of iterations. Defaults to 5
    num_levels (int): number of pyramid levels. Defaults to 5
    scale (float): Scaling factor used to generate the pyramid levels. Defaults to 0.5
    spatial_size (int): size of the support used in the calculation of the standard deviation of the Gaussian
        applicability. Defaults to 9.
    sigma_k (float): scaling factor used to calculate the standard deviation of the Gaussian applicability. The
        formula to calculate sigma is sigma_k*(spatial_size - 1). Defaults to 0.15.
    filter_type (str): Defines the type of filter used to average the calculated matrices. Defaults to "box"
    filter_size (int): Size of the filter used to average the matrices. Defaults to 21
    presmoothing (int): Standard deviation used to perform Gaussian smoothing of the images. Defaults to None
    device_id (int): Device id of the GPU. Defaults to 0
"""
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=11, presmoothing=3, filter_type="gaussian", filter_size=11); RS_sigma = 1.0
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=2, scale=0.5, spatial_size=5, sigma_k=0.5, filter_type="gaussian", filter_size=9, presmoothing=None, device_id=0); RS_sigma = 1.0
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=2, scale=0.5, spatial_size=5, sigma_k=1.0, filter_type="gaussian", filter_size=9, presmoothing=None, device_id=0); RS_sigma = 0.75
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=2, scale=0.5, spatial_size=5, sigma_k=1.0, filter_type="gaussian", filter_size=17, presmoothing=None, device_id=0); RS_sigma = 1.75
#RS_sigma = 1.5
#N_iters = 50
#denoised = denoiser.filter_volume(noisy, std_dev=RS_sigma, N_iters=N_iters, block_size=block_size)

N_iters = 100
RS_std_dev = 1.0
pyramid_levels = 3
window_side = 5
N_poly = 3
iterations = 5 
block_size = (noisy.shape[0], noisy.shape[1], noisy.shape[2])

denoised = denoiser.filter_volume(
    noisy,
    std_dev=RS_std_dev,
    window_side=window_side,
    N_poly=N_poly,
    N_iters=N_iters,
    iterations=iterations,
    pyramid_levels=pyramid_levels,
    block_size=block_size)

In [ ]:
#denoised = RSIVD.filter(farneback, block_size, noisy, RS_sigma=RS_sigma, N_iters=20)

In [ ]:
#np.min(denoised)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
slice_idx = noisy.shape[0]//2
imgplot = plt.imshow(noisy[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
title = f"$N_{{\\mathrm{{iters}}}}={N_iters}, \\sigma_{{\\mathrm{{RS}}}}={RS_std_dev}, w={window_side}, N_{{\\mathrm{{poly}}}}={N_poly}$"
plt.title(title)
plt.imshow(denoised[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[slice_idx][::-1, :] - denoised[slice_idx][::-1, :], cmap="gray")
plt.show()

In [ ]:
fig_single, ax_single = plt.subplots(figsize=(10, 10))
ax_single.imshow(denoised[slice_idx][::-1, :], cmap="gray")
ax_single.set_title(title)
fig_single.savefig(title + ".png")
plt.close(fig_single)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
slice_idx = noisy.shape[1]//2
axs[0].imshow(noisy[:, slice_idx], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, slice_idx], cmap="gray")
axs[1].set_title(f"Denoised")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
slice_idx = noisy.shape[2]//2
axs[0].imshow(noisy[:, :, slice_idx], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, :, slice_idx], cmap="gray")
axs[1].set_title(f"Denoised")
fig.tight_layout()
plt.show()

In [ ]:
with mrcfile.new(f"{fn}_{title}.mrc", overwrite=True) as mrc:
            mrc.set_data(denoised.astype(np.float32))
            mrc.data
#skimage.io.imsave(f"{args.output}_{RS_sigma}_{N_iters}.tif", denoised, imagej=True)

In [ ]:
f"{fn}_{title}.mrc"

In [ ]:
input()

In [ ]:
farneback = opticalflow3D.Farneback3D(iters=5,
                                      num_levels=3,
                                      scale=0.5,
                                      spatial_size=5,
                                      presmoothing=4,
                                      filter_type="box",
                                      filter_size=5,
                                     )

In [ ]:
RS_sigma = 1.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=25)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[75][::-1, :] - denoised_vol[75][::-1, :], cmap="gray")

In [ ]:
skimage.io.imsave(f"{vol_name}_denoised_{RS_sigma}.tif", denoised_vol, imagej=True)